In [4]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import new_alg_v2 as na2
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [92]:
n_s = 30 # number of sample
n_ch = 3 # number of input channels 
d0 = 10 # dim0 of input image
d1 = 10 # dim1 of input image
n_epoch = 1
output_dim = n_s
target = torch.ones(output_dim,1)


In [93]:
# dataset
DS = torch.normal(0, 1, size=(n_s, n_ch, d0, d1))
# לעשות את הרשת כללית מבחינת מימדים וכו

In [94]:
class toy_CNN(nn.Module):
    def __init__(self, n_ch):
        super(toy_CNN, self).__init__()
        self.conv1 = nn.Conv2d(n_ch, 1, kernel_size=5, bias=False)
        self.fc1 = nn.Linear(9, 1, bias=False)

    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = x.view(-1, 9)
        x = self.fc1(x)
        return x

In [95]:
def train(net, eta, n_epoch, dataset, target):
    train_losses = []
    net.train()
    loss = torch.nn.MSELoss()
    optimizer = na2.new_alg(net.parameters(), lr=eta)
    for epoch in range(n_epoch):
        optimizer.zero_grad()
        py_hat = net(dataset)
        objective = loss(py_hat, target)
        train_losses.append(float(objective))
        objective.backward()
        optimizer.step()
    return train_losses

In [96]:
def test(net, dataset, target):
    net.eval()
    loss = torch.nn.MSELoss()
    with torch.no_grad():
        output = net(dataset)
        test_loss = loss(output, target)
    return float(test_loss)

In [97]:
def xl_calc(eta, n_ch):
    network = toy_CNN(n_ch)
    conv1_init = network.conv1.weight.clone()
    fc1_init = network.fc1.weight.clone()
    
    initial_loss = test(network, DS, target)
    test_loss = train(network, eta, n_epoch, DS, target)
    final_loss = test(network, DS, target)
    
    conv1_final = network.conv1.weight.clone()
    fc1_final = network.fc1.weight.clone()
    
    #init weights conv1
    with torch.no_grad():
      network.conv1.weight[:] = conv1_init
    
    loss_conv1 = test(network, DS, target)
    
    #init weights fc1
    with torch.no_grad():
      network.conv1.weight[:] = conv1_final
      network.fc1.weight[:] = fc1_init
    
    loss_fc1 = test(network, DS, target)
    
    dL = final_loss - initial_loss
    print("for lr =",  eta, ":")
    print("  inital loss = ", initial_loss)
    print("  final loss = ", final_loss)
    if dL == 0:
      #print("for lr =",  eta, ":")
      print("   ***dL zero***")
      return
    
    Xl1 = (final_loss - loss_conv1)/dL
    Xl2 = (final_loss-loss_fc1)/dL
    if True:
      #print("for lr =",  eta, ":")
      print("  Xl1 = ", float(Xl1))
      print("  Xl2 = ", float(Xl2))
    return Xl1, Xl2
    

In [98]:
lrs = np.logspace(-2,-5,10)
for lr in lrs:
    xl_calc(lr, n_ch)

for lr = 0.01 :
  inital loss =  1.4154834747314453
  final loss =  0.9628806710243225
  Xl1 =  0.44892620777344583
  Xl2 =  0.4722896088255416
for lr = 0.004641588833612777 :
  inital loss =  1.5622518062591553
  final loss =  1.338186264038086
  Xl1 =  0.47388750147637954
  Xl2 =  0.48928707095879015
for lr = 0.0021544346900318843 :
  inital loss =  1.8761330842971802
  final loss =  1.744376301765442
  Xl1 =  0.48086868562577356
  Xl2 =  0.48850311602017993
for lr = 0.001 :
  inital loss =  1.3713326454162598
  final loss =  1.3346885442733765
  Xl1 =  0.49519019626341526
  Xl2 =  0.4951804367698679
for lr = 0.0004641588833612782 :
  inital loss =  1.0641180276870728
  final loss =  1.0497593879699707
  Xl1 =  0.48945196722264195
  Xl2 =  0.5084392564487874
for lr = 0.00021544346900318845 :
  inital loss =  0.698697030544281
  final loss =  0.6933844685554504
  Xl1 =  0.4995960955907102
  Xl2 =  0.4999102434646023
for lr = 0.0001 :
  inital loss =  0.20605911314487457
  final loss =

In [99]:
network = toy_CNN(n_ch)
conv1_init = network.conv1.weight.clone()
fc1_init = network.fc1.weight.clone()

def xl_calc_db(eta):
    
    #init weights conv1
    with torch.no_grad():
      network.conv1.weight[:] = conv1_init
      network.fc1.weight[:] = fc1_init

    initial_loss = test(network, DS, target)
    test_loss = train(network, eta, n_epoch, DS, target)
    final_loss = test(network, DS, target)
  
    conv1_final = network.conv1.weight.clone()
    fc1_final = network.fc1.weight.clone()
    
    #init weights conv1
    with torch.no_grad():
      network.conv1.weight[:] = conv1_init
    
    loss_conv1 = test(network, DS, target)
    
    #init weights fc1
    with torch.no_grad():
      network.conv1.weight[:] = conv1_final
      network.fc1.weight[:] = fc1_init
    
    loss_fc1 = test(network, DS, target)
    
    dL = final_loss - initial_loss
    print("for lr =",  eta, ":")
    print("  inital loss = ", initial_loss)
    print("  final loss = ", final_loss)
    if dL == 0:
      #print("for lr =",  eta, ":")
      print("   ***dL zero***")
      return
    
    Xl1 = (final_loss - loss_conv1)/dL
    Xl2 = (final_loss-loss_fc1)/dL
    if True:
      #print("for lr =",  eta, ":")
      print("  Xl1 = ", float(Xl1))
      print("  Xl2 = ", float(Xl2))
    return Xl1, Xl2
    
lrs = np.logspace(-2,-5,10)
for lr in lrs:
    xl_calc_db(lr)

for lr = 0.01 :
  inital loss =  1.6686242818832397
  final loss =  1.2367421388626099
  Xl1 =  0.404560171575731
  Xl2 =  0.45579965165931063
for lr = 0.004641588833612777 :
  inital loss =  1.6686242818832397
  final loss =  1.4420363903045654
  Xl1 =  0.4608490196274432
  Xl2 =  0.4747587408595628
for lr = 0.0021544346900318843 :
  inital loss =  1.6686242818832397
  final loss =  1.557746410369873
  Xl1 =  0.483146635186553
  Xl2 =  0.48736333620395844
for lr = 0.001 :
  inital loss =  1.6686242818832397
  final loss =  1.6159354448318481
  Xl1 =  0.49267850112899503
  Xl2 =  0.49371473304584307
for lr = 0.0004641588833612782 :
  inital loss =  1.6686242818832397
  final loss =  1.6439149379730225
  Xl1 =  0.4967459004134564
  Xl2 =  0.4969581767393391
for lr = 0.00021544346900318845 :
  inital loss =  1.6686242818832397
  final loss =  1.6571029424667358
  Xl1 =  0.4984893634632895
  Xl2 =  0.49861352537041637
for lr = 0.0001 :
  inital loss =  1.6686242818832397
  final loss =  1

In [100]:
eta = 0.0021544346900318843
network = toy_CNN(n_ch)

conv1_init = network.conv1.weight.clone()
fc1_init = network.fc1.weight.clone()

initial_loss = test(network, DS, target)
train_loss = train(network, eta, n_epoch, DS, target)
final_loss = test(network, DS, target)

conv1_final = network.conv1.weight.clone()
fc1_final = network.fc1.weight.clone()

#init weights conv1
with torch.no_grad():
  network.conv1.weight[:] = conv1_init

loss_conv1 = test(network, DS, target)

#init weights conv1
with torch.no_grad():
  network.conv1.weight[:] = conv1_final
  network.fc1.weight[:] = fc1_init

loss_fc1 = test(network, DS, target)

dL = final_loss - initial_loss
Xl1 = (final_loss - loss_conv1)/dL
Xl2 = (final_loss-loss_fc1)/dL
print("inital loss = ", initial_loss)
print("final loss = ", final_loss)
print("Xl1 = ", float(Xl1))
print("Xl2 = ", float(Xl2))


inital loss =  1.6914465427398682
final loss =  1.5899059772491455
Xl1 =  0.4827315375729058
Xl2 =  0.4968759685554084
